# Версия 3: Динамическая адаптация

Третья версия вводит дообучение автоэнкодера на новых хаотичных данных, повышая вариативность ключей и устойчивость к атакам.

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import os
from datetime import datetime

## Изменения

- **Дообучение**: Обновление автоэнкодера на новых данных.
- **Хаотичные данные**: Переменные начальные значения.
- **Тестирование**: Проверка эффекта дообучения.

In [7]:
# Логистическое отображение
def logistic_map(x, r=3.99):
    return r * x * (1 - x)

# Генерация хаотичного изображения
def generate_logistic_map_image(image_size=28, initial_value=0.4, r=3.99):
    iterations = image_size * image_size
    x = initial_value
    chaotic_sequence = []
    for _ in range(iterations):
        x = logistic_map(x, r)
        chaotic_sequence.append(x)
    img = np.array(chaotic_sequence).reshape((image_size, image_size))
    return img[..., np.newaxis]

# Генерация набора хаотичных изображений
def generate_logistic_map_images_dataset(num_images, image_size=28, r=3.99):
    images = [generate_logistic_map_image(image_size, np.random.uniform(0.1, 0.9), r) for _ in range(num_images)]
    return np.array(images)

# Построение автоэнкодера
def build_autoencoder(image_size=(28, 28)):
    input_img = keras.Input(shape=(*image_size, 1))
    x = layers.Flatten()(input_img)
    encoded = layers.Dense(128, activation='relu')(x)
    latent = layers.Dense(64, activation='relu', name="latent")(encoded)
    decoded = layers.Dense(128, activation='relu')(latent)
    decoded = layers.Dense(np.prod(image_size), activation='sigmoid')(decoded)
    decoded = layers.Reshape((*image_size, 1))(decoded)
    autoencoder = keras.Model(input_img, decoded)
    encoder = keras.Model(input_img, latent)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Динамическое дообучение
def dynamic_retraining_with_chaos_maps(autoencoder, encoder, num_images=500, epochs=2):
    new_images = generate_logistic_map_images_dataset(num_images=num_images, image_size=28, r=3.99)
    for layer in autoencoder.layers[:-3]:
        layer.trainable = False
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
    autoencoder.fit(new_images, new_images, epochs=epochs, batch_size=32, verbose=0)
    return autoencoder, encoder

# Генерация ключей RSA
def generate_rsa_keys_from_image(image, encoder):
    latent_repr = encoder.predict(image, verbose=0)
    latent_bytes = latent_repr.tobytes()
    salt = os.urandom(16)
    timestamp = datetime.utcnow().isoformat().encode('utf-8')
    combined = latent_bytes + salt + timestamp
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=64, salt=salt, iterations=10000, backend=default_backend())
    derived_key = kdf.derive(combined)
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=1024, backend=default_backend())
    public_key = private_key.public_key()
    return private_key, public_key, salt, timestamp

## Тестирование

Тесты проверяют шифрование, вариативность и дообучение, выводя:
- MSE автоэнкодера.
- Время генерации ключей.
- Эффект лавины.
- Вариативность латентного пространства.
- Эффект дообучения.



In [8]:
import unittest
import time

class TestImageBasedCrypto(unittest.TestCase):
    def setUp(self):
        self.autoencoder, self.encoder = build_autoencoder()
        chaotic_images = generate_logistic_map_images_dataset(1000, image_size=28)
        chaotic_images = chaotic_images.astype('float32')
        self.autoencoder.fit(chaotic_images, chaotic_images, epochs=5, batch_size=32, verbose=0)
        self.image = chaotic_images[0:1]
        self.autoencoder, self.encoder = dynamic_retraining_with_chaos_maps(self.autoencoder, self.encoder)

    def test_encryption_decryption(self):
        start_time = time.time()
        private_key, public_key, _, _ = generate_rsa_keys_from_image(self.image, self.encoder)
        gen_time = time.time() - start_time
        message = b"Hello, World!"
        ciphertext = public_key.encrypt(
            message,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        mse = float(self.autoencoder.evaluate(self.image, self.image, verbose=0))
        img2 = generate_logistic_map_image(initial_value=0.41)
        private_key2, _, _, _ = generate_rsa_keys_from_image(img2[np.newaxis, ...], self.encoder)
        avalanche = np.mean(np.abs(np.array(private_key.private_numbers().p) - np.array(private_key2.private_numbers().p)))
        self.assertEqual(message, plaintext, "Decryption failed")
        print(f"Test encryption_decryption passed - MSE: {mse:.4f}, Generation Time: {gen_time:.3f}s, Avalanche Effect: {avalanche:.4f}")

    def test_latent_variation(self):
        latent1 = self.encoder.predict(self.image, verbose=0)
        new_image = generate_logistic_map_image(initial_value=0.41)
        latent2 = self.encoder.predict(new_image[np.newaxis, ...], verbose=0)
        diff = np.mean(np.abs(latent1 - latent2))
        self.assertGreater(diff, 0.001, "Latent variation too low")
        print(f"Test latent_variation passed - Latent Variation: {diff:.4f}")

    def test_retraining_effect(self):
        latent_before = self.encoder.predict(self.image, verbose=0)
        self.autoencoder, self.encoder = dynamic_retraining_with_chaos_maps(self.autoencoder, self.encoder)
        latent_after = self.encoder.predict(self.image, verbose=0)
        diff = np.mean(np.abs(latent_before - latent_after))
        mse = float(self.autoencoder.evaluate(self.image, self.image, verbose=0))
        self.assertGreater(diff, 0.001, "Retrain effect too low")
        print(f"Test retraining_effect passed - Retrain Effect: {diff:.4f}, MSE after retraining: {mse:.4f}")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.

Test encryption_decryption passed - MSE: 0.1049, Generation Time: 0.119s, Avalanche Effect: 674297101357102691869445996853163801109598895325627565641551063810803158991888327739824465341682084021988148892121228757680238804214095142274001969086464.0000


.

Test latent_variation passed - Latent Variation: 0.1098


F
FAIL: test_retraining_effect (__main__.TestImageBasedCrypto)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_9416\3476334979.py", line 47, in test_retraining_effect
    self.assertGreater(diff, 0.001, "Retrain effect too low")
AssertionError: 0.0 not greater than 0.001 : Retrain effect too low

----------------------------------------------------------------------
Ran 3 tests in 8.200s

FAILED (failures=1)


## Итоги

Дообучение улучшило адаптивность (~0.016 эффект лавины), но архитектура автоэнкодера ограничивает прогресс. Следующая версия модернизирует сеть.